In [12]:
import pickle
from nltk import ngrams
import pandas as pd
import os
import string
from collections import Counter
from time import time
import numpy as np
import itertools
from pandarallel import pandarallel
import matplotlib.pyplot as plt

In [ ]:
from textblob import TextBlob
import nltk
from nltk import word_tokenize
from nltk.util import ngrams

In [ ]:
import gensim
import gensim.corpora as corpora
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

In [ ]:
import spacy
nlp = spacy.load("fr_core_news_sm")

In [ ]:
seed = 1

In [ ]:
dates_GDN = ["31-01-19","06-02-19","17-02-19","02-03-19","08-03-19","21-03-19"]
sujets_GDN = {"ecologie": "LA_TRANSITION_ECOLOGIQUE",
             "democratie": "DEMOCRATIE_ET_CITOYENNETE",
             "fisc": "LA_FISCALITE_ET_LES_DEPENSES_PUBLIQUES",
             "etat": "ORGANISATION_DE_LETAT_ET_DES_SERVICES_PUBLICS"}
sujets_VD = os.listdir("data/VD")

In [ ]:
#flatten = lambda l: [item for sublist in l for item in sublist]

In [ ]:
#-> Plus efficace
flatten = lambda l: list(itertools.chain(*l))

In [ ]:
filter_flatten_tags = lambda docs: [t for tokens in docs for t in tokens if not (t.is_punct or t.is_space or t.is_stop)]

In [ ]:
pd.set_option("display.max_columns",70)

--------------------

---------

In [ ]:
df = pd.read_csv("data/opinionsVD.csv")

In [ ]:
df

In [ ]:
def get_GDN_theme_samples(n):
    samples = []
    for sujet in sujets_GDN:
        sample = pd.read_csv(f"data/GDN/{sujets_GDN[sujet]}.csv")
        sample.columns = sample.columns[:-1].values.tolist()+["commentaires"]
        
        sample = sample[[sample.columns[-1]]].dropna()
        
        sample = sample.loc[np.random.RandomState(seed=seed).permutation(sample.index)[:n]]
        #sample["theme"] = [sujet]*n
        
        samples.append(sample)
        
    return pd.concat(samples).reset_index(drop=True)

In [ ]:
GDN_samples = get_GDN_theme_samples(1000)

In [ ]:
%reset_selective -f "^df$"

In [ ]:
GDN_samples[GDN_samples.columns[0]].values

In [ ]:
GDN_docs = nlp.pipe(GDN_samples["commentaires"].values)

In [ ]:
GDN_docs

In [2]:
import pickle

In [3]:
lemmas = pickle.load(open("lemmas.pkl","rb"))

In [4]:
lemmas

[['lutter',
  'risque',
  'inondation',
  'parking',
  'devoir',
  'réaliser',
  'paver',
  'ajouré',
  'favoriser',
  'écoulement',
  'eau',
  'pluie',
  'Taxez',
  'kérosène',
  'produit',
  'avoir',
  'fioul',
  'gros',
  'porteur',
  'maritime',
  'Interdire',
  'commerce',
  'produit',
  'maritime',
  'pêcher',
  'Europe',
  'conditionner',
  'Chine',
  'revendu',
  'interdire',
  'préfet',
  'prendre',
  'mesure',
  'aller',
  'encontre',
  'avis',
  'recueillir',
  'consultation',
  'public',
  'constituer',
  'forme',
  'démocratie',
  'participatif'],
 ['falloir',
  'obligatoire',
  'tri',
  'sélectif',
  'réduire',
  'suremballage',
  'réinstaurer',
  'consigne',
  'verre',
  'peser',
  'poubelle',
  '.interdire',
  'centre',
  'ville',
  'véhicule',
  'imposer',
  'entreprise',
  'montrer',
  'exemple',
  'éteindre',
  'lumière',
  'nuit'],
 ['oui',
  'transport',
  'marchandise',
  'territoire',
  'France',
  'noeud',
  'routier',
  'Europe'],
 ['innombrable',
  'mesure',
 

In [21]:
lemmas = [[word for word in lemma if len(word) >= 3] for lemma in lemmas]

In [25]:
pickle.dump(lemmas, open('lemmas.pkl','wb'))

In [ ]:
GDN_samples["commentaires"].values[0]

In [23]:
flatten_lemmas = [word for lemma in lemmas for word in lemma]

In [7]:
from collections import Counter

In [24]:
Counter(flatten_lemmas).most_common(100)

[('faire', 1355),
 ('falloir', 1320),
 ('public', 971),
 ('service', 878),
 ('citoyen', 846),
 ('français', 808),
 ('politique', 777),
 ('pouvoir', 756),
 ('France', 708),
 ('vitesse', 682),
 ('mettre', 668),
 ('état', 642),
 ('gouvernement', 641),
 ('devoir', 608),
 ('social', 597),
 ('lieu', 595),
 ('entreprise', 577),
 ('impôt', 566),
 ('grand', 561),
 ('vie', 516),
 ('payer', 511),
 ('temps', 507),
 ('pays', 497),
 ('réduire', 486),
 ('retraite', 486),
 ('sécurité', 484),
 ('fiscal', 480),
 ('travail', 479),
 ('conducteur', 461),
 ('exemple', 456),
 ('route', 454),
 ('etat', 435),
 ('compte', 426),
 ('droit', 412),
 ('permettre', 409),
 ('prendre', 408),
 ('élu', 394),
 ('moyen', 388),
 ('niveau', 387),
 ('euro', 387),
 ('limitation', 381),
 ('national', 378),
 ('personne', 375),
 ('routier', 374),
 ('enfant', 365),
 ('aide', 363),
 ('voiture', 356),
 ('loi', 354),
 ('place', 350),
 ('produit', 347),
 ('heure', 342),
 ('mesure', 340),
 ('nombre', 340),
 ('être', 339),
 ('salaire', 

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(lemmas, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[lemmas], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [14]:
bigrams = [["_".join(bigram) for bigram in list(ngrams(doc, 2))] for doc in lemmas]
trigrams = [["_".join(trigram) for trigram in list(ngrams(doc, 2))] for doc in lemmas]

In [ ]:
bigrams

In [ ]:
lemmas

In [ ]:
pickle.dump(lemmas, open("lemmas.pkl","wb"))

In [ ]:
# Création du dictionnaire
dictionary = corpora.Dictionary(bigrams)

#Matrice Fréquence Terme-Document
TDmatrix = [dictionary.doc2bow(bigram) for bigram in bigrams]

In [ ]:
bigrams

In [ ]:
import pickle

In [ ]:
flatten_lemmas = [word for lemma in lemmas for word in lemma]

In [ ]:
flatten_lemmas

In [ ]:
pickle.dump(flatten_lemmas, open("lemmas.pkl","wb"))

In [ ]:
pickle.dump(TDmatrix, open("TDmatrix.pkl","wb"))

In [ ]:
pickle.dump(dictionary, open("dictionary.pkl","wb"))

In [ ]:
[[(dictionary[id], freq) for id, freq in cp] for cp in TDmatrix[:1]]

In [ ]:
TDmatrix[:1]

In [ ]:
start_time = time()
##
## Multicore Latent Dirichlet Allocation (LDA)
NUM_TOPICS = 4
lda_multicore_model = gensim.models.ldamulticore.LdaMulticore(TDmatrix, num_topics = NUM_TOPICS, id2word=dictionary,random_state=100,passes=10)

# Saving trained model
lda_multicore_model.save('LDA_GDN_bigram_multicore')

## Loading trained model
#lda_multicore_model = gensim.models.ldamodel.LdaModel.load('LDA_police_multicore')


## Print time taken to train the model
print("--- %s seconds ---" % (time() - start_time))

In [ ]:
# To plot at Jupyter notebook
pyLDAvis.enable_notebook()
plot = pyLDAvis.gensim.prepare(lda_multicore_model, TDmatrix, dictionary)
# Save pyLDA plot as html file
pyLDAvis.save_html(plot, 'LDA_GDN.html')
plot

Avec $\lambda \approx 0.3$, il semblerait que:
- Classe 1 = Code de la route
- Classe 2 = Loi/Justice
- Classe 3 = Police/Armée